In [1]:
import torch
import torch.nn as nn
from torch.utils import data
import torchvision
import torchvision.datasets
import sklearn
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np
import mlflow
import mlflow.pytorch

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
batch_size =256
num_classes = 10
learning_rate = 0.001

In [4]:
train_set = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=None)
test_set = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=None)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [5]:
x_train, y_train = train_set.data, train_set.targets
x_test, y_test = test_set.data, test_set.targets

In [6]:
x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1], x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1], x_test.shape[2])

In [7]:
def to_one_hot(num_classes, labels):
    one_hot = torch.zeros(([labels.shape[0], num_classes]))

    for f in range(len(labels)):
        one_hot[f][labels[f]] = 1

    return one_hot

In [10]:
y_train = to_one_hot(num_classes, y_train)
y_test = to_one_hot(num_classes, y_test)

In [17]:
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()

        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1, dilation=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=0, dilation=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=0, dilation=1)
        self.flat1 = nn.Flatten()
        self.dense1 = nn.Linear(256, 128)
        self.dense2 = nn.Linear(128, 64)
        self.dense3 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = self.flat1(x)
        x = self.dense1(x)
        x = nn.ReLU()(x)
        x = self.dense2(x)
        x = nn.ReLU()(x)
        x = self.dense3(x)
        x = nn.Softmax()(x)
        return x

In [18]:
model = model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

In [19]:
dataset = data.TensorDataset(x_train, y_train)
train_loader = data.DataLoader(dataset, batch_size=batch_size)

In [20]:
num_epochs = 5

for f in range(num_epochs):
    for batch_num, minibatch in enumerate(train_loader):
        minibatch_x, minibatch_y = minibatch[0], minibatch[1]

        # cuda
        # output = model.forward(torch.Tensor(minibatch_x.float()).cuda())
        # loss = criterion(output, torch.Tensor(minibatch_y.float()).cuda())

        # cpu
        output = model.forward(torch.Tensor(minibatch_x.float()))
        loss = criterion(output, torch.Tensor(minibatch_y.float()))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"Epoch {f} Batch_Num {batch_num} Loss {loss}")

C:\Users\love9\anaconda3\envs\data\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0 Batch_Num 0 Loss 0.3378298878669739
Epoch 0 Batch_Num 1 Loss 0.3155236840248108
Epoch 0 Batch_Num 2 Loss 0.30136531591415405
Epoch 0 Batch_Num 3 Loss 0.27985188364982605
Epoch 0 Batch_Num 4 Loss 0.26875343918800354
Epoch 0 Batch_Num 5 Loss 0.24444004893302917
Epoch 0 Batch_Num 6 Loss 0.2016797959804535
Epoch 0 Batch_Num 7 Loss 0.1934983879327774
Epoch 0 Batch_Num 8 Loss 0.16755257546901703
Epoch 0 Batch_Num 9 Loss 0.15118944644927979
Epoch 0 Batch_Num 10 Loss 0.1396394670009613
Epoch 0 Batch_Num 11 Loss 0.11607567965984344
Epoch 0 Batch_Num 12 Loss 0.13157711923122406
Epoch 0 Batch_Num 13 Loss 0.1259670853614807
Epoch 0 Batch_Num 14 Loss 0.09769929945468903
Epoch 0 Batch_Num 15 Loss 0.10112454742193222
Epoch 0 Batch_Num 16 Loss 0.10735225677490234
Epoch 0 Batch_Num 17 Loss 0.08644517511129379
Epoch 0 Batch_Num 18 Loss 0.10941453278064728
Epoch 0 Batch_Num 19 Loss 0.09020671993494034
Epoch 0 Batch_Num 20 Loss 0.08545426279306412
Epoch 0 Batch_Num 21 Loss 0.06231990456581116
Epoc

In [21]:
mlflow.set_experiment("PyTorch_MNIST")

with mlflow.start_run():
    preds = model.forward(torch.Tensor(x_test.float()))
    preds = np.round(preds.detach().cpu().numpy())

    eval_acc = accuracy_score(y_test, preds)
    auc_score = roc_auc_score(y_test, preds)

    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_metric("eval_acc", eval_acc)
    mlflow.log_metric("auc_score", auc_score)

    print("eval_acc :", eval_acc)
    print("auc_score :", auc_score)

    mlflow.pytorch.log_model(model, "PyTorch_MNIST")

mlflow.end_run()

2022/03/11 14:20:11 INFO mlflow.tracking.fluent: Experiment with name 'PyTorch_MNIST' does not exist. Creating a new experiment.
2022/03/11 14:20:11 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

C:\Users\love9\anaconda3\envs\data\lib\site-packages\ipykerne

eval_acc : 0.9766
auc_score : 0.9871565095209144


In [24]:
loaded_model = mlflow.pytorch.load_model("runs:/920372197e7e4d149e994bbe1db6f269/PyTorch_MNIST")

In [25]:
preds = loaded_model.forward(torch.Tensor(x_test.float()))
preds = np.round(preds.detach().cpu().numpy())
eval_acc = accuracy_score(y_test, preds)
auc_score = roc_auc_score(y_test, preds)

print("eval_acc :", eval_acc)
print("auc_score :", auc_score)

eval_acc : 0.9766
auc_score : 0.9871565095209144


C:\Users\love9\anaconda3\envs\data\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
